J'ai mal à la tête et je me sens fatigué.

Bonjour docteur, je ressens une douleur intense à la poitrine depuis hier. J'ai également des maux de tête fréquents et une sensation de vertige. En plus, j'ai des difficultés à respirer correctement et une toux sèche qui persiste. Pourriez-vous m'aider à comprendre ce que j'ai ?

In [4]:
!pip install openai==0.28

In [2]:
import pandas as pd
import json
import ast

# Load the global symptoms DataFrame
global_symptoms_path = 'global_symptoms.csv'
global_symptoms_df = pd.read_csv(global_symptoms_path)

# Create a dictionary to map each symptom to a list of unique associated feature names
symptom_to_features = {}

for index, row in global_symptoms_df.iterrows():
    symptom = row['Symptom Name']
    # Clean and parse the Features string manually
    features_str = row['Features'].strip("[]").replace("'", "").replace(" ", "")
    features = features_str.split(',')
    
    if symptom in symptom_to_features:
        symptom_to_features[symptom].update(features)
    else:
        symptom_to_features[symptom] = set(features)

# Convert sets to lists for JSON compatibility
symptom_to_features = {symptom: list(features) for symptom, features in symptom_to_features.items()}

json_result = symptom_to_features

In [4]:
import openai
import pandas as pd
import gradio as gr
import json
import requests

openai.api_key = api_key

# Load your symptom, feature, and risk factor list
symptom_df = pd.read_csv('symptom_ids.csv')
feature_df = pd.read_csv('feature_ids.csv')
factors_df = pd.read_csv('factors_ids.csv')

# Create dictionaries to map symptom, feature, and risk factor names to their IDs
symptom_name_to_id = dict(zip(symptom_df['symptom_name'].str.lower(), symptom_df['symptom_id']))
feature_name_to_id = dict(zip(feature_df['feature_name'].str.lower(), feature_df['feature_id']))
factor_name_to_id = dict(zip(factors_df['factor_name'].str.lower(), factors_df['factor_id']))

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

def extract_symptoms_from_message(message):
    prompt = f"Extract all symptoms from the following message: \"{message}\". List each symptom separated by a comma."
    result = get_completion(prompt)
    print(f"Extracted Symptoms: {result}")
    return result

def translate_symptoms_to_english(symptoms):
    prompt = f"Translate these symptoms from French to English: {symptoms}. List each symptom separated by a comma."
    result = get_completion(prompt)
    print(f"Translated Symptoms: {result}")
    return result

def map_symptoms_with_gpt(translated_symptoms, symptom_list):
    symptom_list_str = ", ".join(symptom_list)
    prompt = (
        f"The extracted symptoms are: {translated_symptoms}.\n"
        f"Map each extracted symptom to the closest match from the following symptom list:\n"
        f"{symptom_list_str}\n"
        f"Ensure that each extracted symptom is mapped to only one symptom from the list.\n"
        f"Format the output in a JSON format like this: "
        f"{{\"extracted_symptom_1\": \"mapped_symptom_1\", \"extracted_symptom_1_1\": \"mapped_symptom_1_2\", \"extracted_symptom_2\": \"mapped_symptom_2_1\", \"extracted_symptom_2_2\": \"mapped_symptom_2_2\"}}."
    )
    result = get_completion(prompt)
    print(f"Mapped Symptoms: {result}")
    return result

def extract_features(translated_symptoms, mapped_symptoms_dict):
    features_dict = {}
    for extracted_symptom, mapped_symptom in mapped_symptoms_dict.items():
        possible_features = json_result.get(mapped_symptom, [])[1:]
        if possible_features:
            feature_prompt = f"From the symptom description '{translated_symptoms}', extract which of the following features are present: {', '.join(possible_features)}."
            features = get_completion(feature_prompt)
            # Ensure collected features are in the possible features list
            collected_features = [feature.strip().lower() for feature in features.split(',') if feature.strip().lower() in possible_features]
            if collected_features:
                features_dict[extracted_symptom] = collected_features
            else:
                features_dict[extracted_symptom] = []
        else:
            features_dict[extracted_symptom] = []
    return features_dict

def map_features_with_ids(features_dict):
    mapped_features_with_ids = {}
    for symptom, features in features_dict.items():
        feature_ids = [feature_name_to_id.get(feature, "ID not found") for feature in features]
        mapped_features_with_ids[symptom] = feature_ids
    return mapped_features_with_ids

def translate_and_map_risk_factors(risk_factors):
    translated_risk_factors = translate_symptoms_to_english(risk_factors)
    factor_list_str = ", ".join(factor_name_to_id.keys())
    prompt = (
        f"The extracted risk factors are: {translated_risk_factors}.\n"
        f"Map each extracted risk factor to the closest match from the following factor list:\n"
        f"{factor_list_str}\n"
        f"Ensure that each extracted risk factor is mapped to only one factor from the list.\n"
        f"Format the output in a JSON format like this: "
        f"{{\"extracted_risk_factor_1\": \"mapped_risk_factor_1\", \"extracted_risk_factor_2\": \"mapped_risk_factor_2\"}}."
    )
    result = get_completion(prompt)
    print(f"Mapped Risk Factors: {result}")
    mapped_risk_factors_dict = json.loads(result)
    mapped_risk_factors_dict = {k.lower(): v.lower() for k, v in mapped_risk_factors_dict.items()}
    mapped_risk_factors_ids = [factor_name_to_id.get(mapped_risk_factors_dict[factor], "ID not found") for factor in mapped_risk_factors_dict]
    return mapped_risk_factors_ids

def create_session(age, gender, symptoms_with_ids_and_features, risk_factors_ids):
    base_url = 'http://127.0.0.1:8000'
    create_session_url = f'{base_url}/create_session'

    data = {
        "symptoms_dict": symptoms_with_ids_and_features,
        "risk_factors": risk_factors_ids,
        "age": age,
        "gender": gender,
        "weight_rf": 0.4,
        "weight_rf_f": 0.1,
        "weight_match": 0.4,
        "weight_disease_score": 0.1,
        "penalty_factor": 0.5,
        "threshold_probability": 0.8,
        "threshold_difference": 0.3
    }
    
    print(f"Data for Creating Session: {data}")
    
    response = requests.post(create_session_url, json=data)
    result = response.json()
    return result

def update_session(session_id, selected_symptoms_ids, selected_features, rejected_symptoms, risk_factors_ids):
    base_url = 'http://127.0.0.1:8000'
    update_session_url = f'{base_url}/update_session'

    update_data = {
        "session_id": session_id,
        "selected_symptoms": selected_symptoms_ids,
        "selected_features": selected_features,
        "selected_risk_factors": risk_factors_ids,
        "rejected_symptoms": rejected_symptoms
    }
    
    print(f"Data for Updating Session: {update_data}")
    
    response = requests.post(update_session_url, json=update_data)
    result = response.json()
    return result

def chatbot(patient_message, age, gender, risk_factors_input):
    # Step 1: Extract symptoms from the patient message
    extracted_symptoms = extract_symptoms_from_message(patient_message)

    # Step 2: Translate extracted symptoms to English
    translated_symptoms = translate_symptoms_to_english(extracted_symptoms)

    # Step 3: Map the translated symptoms to your English symptom list using GPT
    english_symptom_list = symptom_df['symptom_name'].str.lower().tolist()
    mapped_symptoms_response = map_symptoms_with_gpt(translated_symptoms, english_symptom_list)

    # Convert the GPT response to a dictionary
    mapped_symptoms_dict = json.loads(mapped_symptoms_response)
    mapped_symptoms_dict = {k.lower(): v.lower() for k, v in mapped_symptoms_dict.items()}
    print(f"Mapped Symptoms Dictionary: {mapped_symptoms_dict}")

    # Extract IDs for mapped symptoms
    mapped_symptoms_with_ids = {symptom: symptom_name_to_id.get(mapped_symptoms_dict[symptom], "ID not found") for symptom in mapped_symptoms_dict}
    print(f"Mapped Symptoms with IDs: {mapped_symptoms_with_ids}")

    # Extract features for each mapped symptom
    features_dict = extract_features(translated_symptoms, mapped_symptoms_dict)

    # Map features to their IDs
    mapped_features_with_ids = map_features_with_ids(features_dict)
    print(f"Mapped Features with IDs: {mapped_features_with_ids}")

    # Ensure all symptoms are included, even those without features
    symptoms_with_ids_and_features = {mapped_symptoms_with_ids[symptom]: mapped_features_with_ids.get(symptom, []) for symptom in mapped_symptoms_with_ids}
    
    # Translate and map risk factors
    risk_factors_ids = []
    if risk_factors_input:
        risk_factors_ids = translate_and_map_risk_factors(risk_factors_input)
    
    # Create a session with the collected data
    session_result = create_session(age, gender, symptoms_with_ids_and_features, risk_factors_ids)
    
    final_output = {
        "extracted_symptoms": extracted_symptoms,
        "mapped_symptoms_with_ids": mapped_symptoms_with_ids,
        "features": mapped_features_with_ids,
        "session_result": session_result
    }

    print(f"Final Mapped Symptoms with IDs and Features: {final_output}")
    return final_output

def format_session_result(session_result):
    session_id = session_result['session_id']
    predicted_diseases = "\n".join([f"- {disease}: {probability:.2f}" for disease, probability in session_result['predicted_diseases']])
    top_3_symptoms = "\n".join([f"- {symptom}" for symptom in session_result['top_3_symptoms']])
    top_symptom_features = "\n".join([f"- {symptom}: {', '.join(features)}" for symptom, features in session_result['top_symptom_features'].items()])
    diagnosis_complete = "Yes" if session_result['diagnosis_complete'] else "No"
    
    formatted_result = (
        f"**Session ID:** {session_id}\n\n"
        f"**Predicted Diseases:**\n{predicted_diseases}\n\n"
        f"**Top 3 Symptoms to Ask About Next:**\n{top_3_symptoms}\n\n"
        f"**Top Symptom Features:**\n{top_symptom_features}\n\n"
        f"**Diagnosis Complete:** {diagnosis_complete}"
    )
    
    return formatted_result

def process_update(session_id, message, rejected_symptoms_input, risk_factors_input):
    # Extract symptoms from the patient message
    extracted_symptoms = extract_symptoms_from_message(message)

    # Translate extracted_symptoms to English
    translated_symptoms = translate_symptoms_to_english(extracted_symptoms)

    # Map the translated symptoms to your English symptom list using GPT
    english_symptom_list = symptom_df['symptom_name'].str.lower().tolist()
    mapped_symptoms_response = map_symptoms_with_gpt(translated_symptoms, english_symptom_list)

    # Convert the GPT response to a dictionary
    mapped_symptoms_dict = json.loads(mapped_symptoms_response)
    mapped_symptoms_dict = {k.lower(): v.lower() for k, v in mapped_symptoms_dict.items()}

    # Extract IDs for mapped symptoms
    selected_symptoms_ids = [symptom_name_to_id.get(mapped_symptoms_dict[symptom], "ID not found") for symptom in mapped_symptoms_dict]

    # Extract features for each mapped symptom
    features_dict = extract_features(translated_symptoms, mapped_symptoms_dict)

    # Map features to their IDs
    mapped_features_with_ids = map_features_with_ids(features_dict)
    selected_features = {symptom_name_to_id.get(symptom, "ID not found"): features for symptom, features in mapped_features_with_ids.items()}

    # Translate and map rejected symptoms if provided
    rejected_symptoms = []
    if rejected_symptoms_input:
        translated_rejected_symptoms = translate_symptoms_to_english(rejected_symptoms_input)
        mapped_rejected_symptoms_response = map_symptoms_with_gpt(translated_rejected_symptoms, english_symptom_list)
        mapped_rejected_symptoms_dict = json.loads(mapped_rejected_symptoms_response)
        mapped_rejected_symptoms_dict = {k.lower(): v.lower() for k, v in mapped_rejected_symptoms_dict.items()}
        rejected_symptoms = [symptom_name_to_id.get(mapped_rejected_symptoms_dict[symptom], "ID not found") for symptom in mapped_rejected_symptoms_dict]

    # Translate and map risk factors if provided
    risk_factors_ids = []
    if risk_factors_input:
        risk_factors_ids = translate_and_map_risk_factors(risk_factors_input)

    # Update the session with the new symptoms, features, rejected symptoms, and risk factors
    update_result = update_session(session_id, selected_symptoms_ids, selected_features, rejected_symptoms, risk_factors_ids)
    
    formatted_result = format_session_result(update_result)
    
    new_history = (
        f"**Extracted Symptoms (French):** {extracted_symptoms}\n\n{formatted_result}"
    )
    
    return update_result, new_history

def chat_flow(patient_message, session_id, age, gender, history, rejected_symptoms_input, risk_factors_input):
    if not session_id:
        # Create a new session
        session_data = chatbot(patient_message, age, gender, risk_factors_input)
        session_result = session_data['session_result']
        formatted_result = format_session_result(session_result)
        session_id = session_result['session_id']
        new_history = (
            f"**Extracted Symptoms (French):** {session_data['extracted_symptoms']}\n\n{formatted_result}"
        )
    else:
        # Update the existing session
        update_result, new_history = process_update(session_id, patient_message, rejected_symptoms_input, risk_factors_input)

    return new_history, session_id

with gr.Blocks() as demo:
    age_input = gr.Slider(0, 100, label="Enter your age")
    gender_input = gr.Radio(["male", "female"], label="Select your gender")
    start_button = gr.Button("Start Chat")
    chatbot_component = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    session_id = gr.State(None)
    rejected_symptoms_input = gr.Textbox(label="Rejected Symptoms (comma-separated, in French)")
    risk_factors_input = gr.Textbox(label="Risk Factors (comma-separated, in French)")

    def start_chat(age, gender):
        return [(None, f"Age: {age}, Gender: {gender}. Please describe your symptoms:")], age, gender

    def user(user_message, history, age, gender, session_id):
        return "", history + [[user_message, None]], age, gender, session_id

    def bot(history, age, gender, session_id, rejected_symptoms_input, risk_factors_input):
        user_message, _ = history[-1]
        new_history, session_id = chat_flow(user_message, session_id, age, gender, history, rejected_symptoms_input, risk_factors_input)
        
        history[-1][1] = new_history
        return history, age, gender, session_id, rejected_symptoms_input, risk_factors_input

    start_button.click(
        start_chat, [age_input, gender_input], [chatbot_component, age_input, gender_input]
    )
    msg.submit(user, [msg, chatbot_component, age_input, gender_input, session_id], [msg, chatbot_component, age_input, gender_input, session_id]).then(
        bot, [chatbot_component, age_input, gender_input, session_id, rejected_symptoms_input, risk_factors_input], [chatbot_component, age_input, gender_input, session_id, rejected_symptoms_input, risk_factors_input]
    )
    clear.click(lambda: None, None, chatbot_component, queue=False)

demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Extracted Symptoms: fatigue, fièvre, douleurs musculaires
Translated Symptoms: fatigue, fever, muscle pain
Mapped Symptoms: {
    "fatigue": "tiredness",
    "fever": "fever",
    "muscle pain": "muscle pain"
}
Mapped Symptoms Dictionary: {'fatigue': 'tiredness', 'fever': 'fever', 'muscle pain': 'muscle pain'}
Mapped Symptoms with IDs: {'fatigue': 'S011', 'fever': 'S024', 'muscle pain': 'S137'}
Mapped Features with IDs: {'fatigue': [], 'fever': ['F073', 'F046', 'F054'], 'muscle pain': ['F013', 'F054', 'F046', 'F009', 'ID not found', 'F013', 'F106', 'F054', 'F046', 'F009', 'F092', 'F106', 'F054', 'F046', 'F009', 'F092', 'F055', 'F093', 'F024']}
Data for Creating Session: {'symptoms_dict': {'S011': [], 'S024': ['F073', 'F046', 'F054'], 'S137': ['F013', 'F054', 'F046', 'F009', 'ID not found', 'F013', 'F106', 'F054', 'F046', 'F009', 'F092', 'F106', 'F054', 'F046', 'F009', 'F092', 'F055', 'F093', 'F024']}, 'risk_factors': [], 'age': 31, 'gender': 'male', 'weight_rf': 0.4, 'weight_rf_f': 0.1